# Lab6 Introduction Transformers

## Cài đặt
cài đặt thư viện transformers và torch.

```bash
pip install transformers torch
```

# Bài tập thực hành

## Bài 1. Khôi phục Masked Token

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM

# 1. Tải pipeline "fill-mask" với mô hình BERT
model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

mask_filler = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer
)

# 2. Câu đầu vào với token [MASK]   
input_sentence = "Hanoi is the [MASK] of Vietnam."

# 3. Thực hiện dự đoán
# top_k=5 yêu cầu mô hình trả về 5 dự đoán hàng đầu
predictions = mask_filler(input_sentence, top_k=5)

# 4. In kết quả
print(f"Câu gốc: {input_sentence}")

for pred in predictions:
    print(f"Dự đoán: '{pred['token_str']}' với độ tin cậy: {pred['score']:.4f}")
    print(f" -> Câu hoàn chỉnh: {pred['sequence']}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
Device set to use cpu


Câu gốc: Hanoi is the [MASK] of Vietnam.
Dự đoán: 'capital' với độ tin cậy: 0.9991
 -> Câu hoàn chỉnh: hanoi is the capital of vietnam.
Dự đoán: 'center' với độ tin cậy: 0.0001
 -> Câu hoàn chỉnh: hanoi is the center of vietnam.
Dự đoán: 'birthplace' với độ tin cậy: 0.0001
 -> Câu hoàn chỉnh: hanoi is the birthplace of vietnam.
Dự đoán: 'headquarters' với độ tin cậy: 0.0001
 -> Câu hoàn chỉnh: hanoi is the headquarters of vietnam.
Dự đoán: 'city' với độ tin cậy: 0.0001
 -> Câu hoàn chỉnh: hanoi is the city of vietnam.


### Câu hỏi Bài 1:

**1. Mô hình đã dự đoán đúng từ "capital" không?**

*Trả lời:* Có, mô hình BERT đã dự đoán đúng từ "capital" với độ tin cậy cao nhất trong top 5 kết quả. Điều này cho thấy mô hình đã học được mối quan hệ ngữ nghĩa giữa "Hanoi", "Vietnam" và khái niệm "thủ đô".

**2. Tại sao các mô hình Encoder-only như BERT lại phù hợp cho tác vụ này?**

*Trả lời:* Các mô hình Encoder-only như BERT phù hợp cho tác vụ Masked Language Modeling vì:
- BERT có khả năng nhìn bidirectional (hai chiều), có thể xem xét cả ngữ cảnh trước và sau token bị mask, giúp dự đoán chính xác hơn.
- BERT được huấn luyện đặc biệt cho tác vụ MLM trong quá trình pre-training, nên nó rất giỏi trong việc dự đoán từ bị thiếu dựa trên ngữ cảnh xung quanh.
- Cơ chế Self-Attention cho phép mô hình nắm bắt được mối quan hệ ngữ nghĩa và ngữ pháp phức tạp giữa các từ trong câu.

## Bài 2: Dự đoán từ tiếp theo

In [2]:
from transformers import pipeline

# 1. Tải pipeline "text-generation"
generator = pipeline("text-generation", model="openai-community/gpt2")

# 2. Đoạn văn bản mồi
prompt = "The best thing about learning NLP is"

# 3. Sinh văn bản
# max_length: tổng độ dài của câu mồi và phần được sinh ra
# num_return_sequences: số lượng chuỗi kết quả muốn nhận
generated_texts = generator(prompt, max_length=50, num_return_sequences=1)

# 4. In kết quả
print(f"Câu mồi: '{prompt}'")

for text in generated_texts:
    print("Văn bản được sinh ra:")
    print(text['generated_text'])

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you c

Câu mồi: 'The best thing about learning NLP is'
Văn bản được sinh ra:
The best thing about learning NLP is that it shows you how to build trust and trust with your collaborators.

The best thing about learning NLP is that it shows you how to build trust and trust with your collaborators. You don't need a lot of hands-on experience to make this work. You can learn from people who have worked on it and get it in your hands.

You don't need a lot of hands-on experience to make this work. You can learn from people who have worked on it and get it in your hands. It's often easy to feel like you've got a huge amount of other people to help you with your research. You can learn from people who've written and written about it and got it out there.

It's often easy to feel like you've got a huge amount of other people to help you with your research. You can learn from people who've written and written about it and got it out there. You can learn from people who've worked on the project and have

### Câu hỏi Bài 2:

**1. Kết quả sinh ra có hợp lý không?**

*Trả lời:* Có, kết quả sinh ra khá hợp lý và mạch lạc. GPT-2 đã tạo ra một đoạn văn bản tiếp nối câu mồi một cách tự nhiên, với ngữ pháp đúng và nội dung có ý nghĩa liên quan đến việc học NLP.

**2. Tại sao các mô hình Decoder-only như GPT lại phù hợp cho tác vụ này?**

*Trả lời:* Các mô hình Decoder-only như GPT phù hợp cho tác vụ text generation vì:
- GPT có khả năng nhìn unidirectional (một chiều), chỉ xem xét các token đứng trước, phù hợp với bản chất tuần tự của việc sinh văn bản.
- GPT được huấn luyện cho tác vụ Next Token Prediction, tức là dự đoán token tiếp theo dựa trên các token đã xuất hiện trước đó.
- Kiến trúc autoregressive (tự hồi quy) của GPT cho phép sinh văn bản dài và mạch lạc, với mỗi token mới được sinh ra dựa trên toàn bộ chuỗi trước đó.
- Phù hợp với tác vụ sinh văn bản vì không cần biết trước các token phía sau (như trong BERT).

## Bài 3: Tính toán Vector biểu diễn của câu

In [3]:
import torch
from transformers import AutoTokenizer, AutoModel

# 1. Chọn một mô hình BERT
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 2. Câu đầu vào
sentences = ["This is a sample sentence."]

# 3. Tokenize câu
# padding=True: đệm các câu ngắn hơn để có cùng độ dài
# truncation=True: cắt các câu dài hơn
# return_tensors='pt': trả về kết quả dưới dạng PyTorch tensors
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# 4. Đưa qua mô hình để lấy hidden states
# torch.no_grad() để không tính toán gradient, tiết kiệm bộ nhớ
with torch.no_grad():
    outputs = model(**inputs)
# outputs.last_hidden_state chứa vector đầu ra của tất cả các token

last_hidden_state = outputs.last_hidden_state
# shape: (batch_size, sequence_length, hidden_size)

# 5. Thực hiện Mean Pooling
# Để tính trung bình chính xác, chúng ta cần bỏ qua các token đệm (padding tokens)
attention_mask = inputs['attention_mask']
mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)
sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
sentence_embedding = sum_embeddings / sum_mask

# 6. In kết quả
print("Vector biểu diễn của câu:")
print(sentence_embedding)
print("\nKích thước của vector:", sentence_embedding.shape)

Vector biểu diễn của câu:
tensor([[-6.3874e-02, -4.2837e-01, -6.6779e-02, -3.8430e-01, -6.5785e-02,
         -2.1826e-01,  4.7636e-01,  4.8659e-01,  3.9991e-05, -7.4274e-02,
         -7.4740e-02, -4.7635e-01, -1.9773e-01,  2.4824e-01, -1.2162e-01,
          1.6678e-01,  2.1045e-01, -1.4576e-01,  1.2637e-01,  1.8636e-02,
          2.4640e-01,  5.7090e-01, -4.7014e-01,  1.3782e-01,  7.3650e-01,
         -3.3808e-01, -5.0329e-02, -1.6453e-01, -4.3517e-01, -1.2900e-01,
          1.6516e-01,  3.4004e-01, -1.4930e-01,  2.2422e-02, -1.0488e-01,
         -5.1916e-01,  3.2964e-01, -2.2162e-01, -3.4206e-01,  1.1993e-01,
         -7.0148e-01, -2.3126e-01,  1.1224e-01,  1.2550e-01, -2.5191e-01,
         -4.6374e-01, -2.7261e-02, -2.8415e-01, -9.9250e-02, -3.7018e-02,
         -8.9192e-01,  2.5005e-01,  1.5816e-01,  2.2701e-01, -2.8497e-01,
          4.5300e-01,  5.0922e-03, -7.9441e-01, -3.1008e-01, -1.7403e-01,
          4.3029e-01,  1.6816e-01,  1.0590e-01, -4.8987e-01,  3.1856e-01,
          3.

### Câu hỏi Bài 3:

**1. Kích thước của vector biểu diễn là bao nhiêu? Con số này tương ứng với tham số nào của mô hình BERT?**

*Trả lời:* Kích thước của vector biểu diễn là 768 (đối với mô hình `bert-base-uncased`). Con số này tương ứng với tham số hidden_size của mô hình BERT, là kích thước của hidden states ở mỗi layer trong kiến trúc Transformer. 


**2. Tại sao chúng ta cần sử dụng attention_mask khi thực hiện Mean Pooling?**

*Trả lời:* Chúng ta cần sử dụng `attention_mask` khi thực hiện Mean Pooling vì:
- Attention_mask xác định vị trí nào là token thật (giá trị 1) và vị trí nào là padding token (giá trị 0).
- Khi tính trung bình, ta chỉ muốn tính trên các token thật, không tính các padding token.
- Nếu không dùng mask, các padding token (có giá trị embedding) sẽ làm sai lệch giá trị trung bình, dẫn đến vector biểu diễn không chính xác.
- Đảm bảo vector biểu diễn chỉ phản ánh ngữ nghĩa của câu thực, không bị ảnh hưởng bởi các token đệm không mang nghĩa.
- Điều này đặc biệt quan trọng khi xử lý batch với các câu có độ dài khác nhau.